# Pré-processamento de Texto em Python

Existem diversas etapas diferentes envolvendo o pré-processamento de um texto em python. A aplicação em que será empregado pode gerar variações na ordem destas etapas, bem como possibilitar que algumas sejam omitidas. Neste _notebook_ exploraremos uma alternativa para cumprir a tarefa de forma mais genérica.

1. [Remoção de tags HTML](#Remo%C3%A7%C3%A3o-de-tags-HTML)
1. [Tratamento-para-Acentuação](#Tratamento-para-Acentua%C3%A7%C3%A3o)
1. [Tokenização](#Tokeniza%C3%A7%C3%A3o)
1. [Tratamento para Números](#Tratamento-para-N%C3%BAmeros)
1. [Remoção de _Stop Words_](#Remo%C3%A7%C3%A3o-de-Stop-Words)
1. [Lemmatização e _Stemming_](#Lematiza%C3%A7%C3%A3o-e-Stemming)

[Referências](#Refer%C3%AAncias)

## Preparando o Dataset

Começaremos carregando um dataset para exemplificarmos as decisões a serem tomadas. O dataset em questão é o _[COVID-19 News Articles Open Research Dataset](https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26)_.

In [4]:
import pandas as pd

raw_dataset = pd.read_csv("../datasets/cbc-news-coronavirus/news.csv", nrows=15)

# Apenas a coluna com o conteúdo das notícias nos interessa agora
dataset = pd.DataFrame(raw_dataset["text"])

# Início do dataset completo
raw_dataset.head()

,Unnamed: 0,authors,title,publish_date,description,text,url
0,0,[],'More vital now:' Gay-straight alliances go vi...,2020-05-03 1:30,Lily Overacker and Laurell Pallot start each g...,Lily Overacker and Laurell Pallot start each g...,https://www.cbc.ca/news/canada/calgary/gay-str...
1,1,[],Scientists aim to 'see' invisible transmission...,2020-05-02 8:00,Some researchers aim to learn more about how t...,"This is an excerpt from Second Opinion, a week...",https://www.cbc.ca/news/technology/droplet-tra...
2,2,['The Canadian Press'],Coronavirus: What's happening in Canada and ar...,2020-05-02 11:28,Canada's chief public health officer struck an...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
3,3,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
4,4,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...


In [5]:
# Início do dataset resumido
dataset.head()

,text
0,Lily Overacker and Laurell Pallot start each g...
1,"This is an excerpt from Second Opinion, a week..."
2,The latest: The lives behind the numbers: Wha...
3,B.C. provincial health officer Dr. Bonnie Henr...
4,B.C. provincial health officer Dr. Bonnie Henr...


In [6]:
corpus = [document for document in dataset["text"]]

# Seleciona apenas o terceiro documento
document = corpus[2]

print(u"{}".format(document))

The latest:  The lives behind the numbers: What we know about the first 1,000 COVID-19 deaths in Canada. Health expert urges provinces to 'find the middle ground' as they begin to reopen. Canadians have lost more than $1.2 million to COVID-19 scams. Canada is officially in a recession, C.D. Howe Institute says. Major U.S. airlines to require passengers to wear masks. Nova Scotia is easing some of its COVID-19 restrictions. Ontario allowing some businesses to reopen under strict guidelines. EI claimants are going weeks without income as federal call system slows to a crawl. INTERACTIVE | See the latest data on coronavirus cases in Canada. Have a coronavirus question or news tip for CBC News? Email: Covid@cbc.ca  Canada's chief public health officer struck an optimistic tone about the COVID-19 pandemic on Saturday, saying that efforts to flatten the curve are working as provinces across the country reported more positive figures. "By following public health recommendations, we have colle

In [7]:
# Exemplo de Tag HTML
html_txt = document[12244:12550]

html_txt

'<br><br>Read the full story: <a href="https://t.co/2waGuP1Oc2">https://t.co/2waGuP1Oc2</a><a href="https://twitter.com/hashtag/Covid19nfld?src=hash&amp;ref_src=twsrc%5Etfw">#Covid19nfld</a> <a href="https://t.co/Vi92XMwNTv">pic.twitter.com/Vi92XMwNTv</a>&mdash;@CBCNL In Canada\'s North,\xa0\xa0all of Yukon\'s 11 '

In [8]:
# Exemplo de Número
num_txt = document[:249]

num_txt

"The latest:  The lives behind the numbers: What we know about the first 1,000 COVID-19 deaths in Canada. Health expert urges provinces to 'find the middle ground' as they begin to reopen. Canadians have lost more than $1.2 million to COVID-19 scams."

In [9]:
# Exemplo de texto com acentuação
acc_txt = "Úá à éó ão ção üä íï"

acc_txt

'Úá à éó ão ção üä íï'

In [10]:
# Concatena todos os exemplos para termos um texto rico em "lixo"
bad_txt = " ".join([html_txt, num_txt, acc_txt])

bad_txt

'<br><br>Read the full story: <a href="https://t.co/2waGuP1Oc2">https://t.co/2waGuP1Oc2</a><a href="https://twitter.com/hashtag/Covid19nfld?src=hash&amp;ref_src=twsrc%5Etfw">#Covid19nfld</a> <a href="https://t.co/Vi92XMwNTv">pic.twitter.com/Vi92XMwNTv</a>&mdash;@CBCNL In Canada\'s North,\xa0\xa0all of Yukon\'s 11  The latest:  The lives behind the numbers: What we know about the first 1,000 COVID-19 deaths in Canada. Health expert urges provinces to \'find the middle ground\' as they begin to reopen. Canadians have lost more than $1.2 million to COVID-19 scams. Úá à éó ão ção üä íï'

## Remoção de tags HTML

Muitos textos provém de _crawlers_ que acabam retornando muitos "lixos" junto com o material de interesse. Um destes "lixos" são tags HTML.

Remove apenas as tags HTML (incluindo parâmetros), mantendo o texto contido nelas:

In [11]:
from bs4 import BeautifulSoup


def strip_html_tags(text, separator=""):
    parser = BeautifulSoup(text, "html.parser")
    
    return parser.get_text(separator=separator)


strip_html_tags(html_txt)

"Read the full story: https://t.co/2waGuP1Oc2#Covid19nfld pic.twitter.com/Vi92XMwNTv—@CBCNL In Canada's North,\xa0\xa0all of Yukon's 11 "

Remove as tags e o conteúdo:

In [12]:
import re


def delete_html_nodes(text):
    regex = re.compile("<.+>")
    
    return re.sub(regex, "", text)


delete_html_nodes(html_txt)

"&mdash;@CBCNL In Canada's North,\xa0\xa0all of Yukon's 11 "

## Tratamento para Acentuação

Caracteres acentuados como `à`, `ä` e `é` introduzem distinções (por exemplo `a` $\neq$ `à` $\neq$ `á`) indesejadas, que em etapas posteriores ao pré-processamento resultarão em problemas maiores.

Para evitar isto, basta remover estas acentuações. Uma forma simples de fazê-lo é converter caracteres no formato unicode para o formato ascii.

In [13]:
import unidecode

def utf8_to_ascii(text):
    return unidecode.unidecode(text)


utf8_to_ascii(acc_txt)

'Ua a eo ao cao ua ii'

## Tokenização

A tokenização é o processo de dividir um documento em unidades menores, chamadas tokens. A divisão ocorre nos locais onde um termo começa e outro termina, como por exemplo espaços em branco, potuação, etc.

In [14]:
import spacy
spacy.cli.download("en_core_web_md")

def tokenize(corpus):
    nlp = spacy.load("en_core_web_md")
    
    tokens = []
    for document in corpus:
        spacy_doc = nlp(document)
        
        tokens.append([token for token in spacy_doc])
            
    return tokens


tokenize([bad_txt])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


[[<,
  br><br,
  >,
  Read,
  the,
  full,
  story,
  :,
  <,
  a,
  href="https://t.co/2waGuP1Oc2">https://t.co/2waGuP1Oc2</a><a,
  href="https://twitter.com,
  /,
  hashtag,
  /,
  Covid19nfld?src,
  =,
  hash&amp;ref_src,
  =,
  twsrc%5Etfw">#Covid19nfld</a,
  >,
  <,
  a,
  href="https://t.co,
  /,
  Vi92XMwNTv">pic.twitter.com,
  /,
  Vi92XMwNTv</a>&mdash;@CBCNL,
  In,
  Canada,
  's,
  North,
  ,,
    ,
  all,
  of,
  Yukon,
  's,
  11,
   ,
  The,
  latest,
  :,
   ,
  The,
  lives,
  behind,
  the,
  numbers,
  :,
  What,
  we,
  know,
  about,
  the,
  first,
  1,000,
  COVID-19,
  deaths,
  in,
  Canada,
  .,
  Health,
  expert,
  urges,
  provinces,
  to,
  ',
  find,
  the,
  middle,
  ground,
  ',
  as,
  they,
  begin,
  to,
  reopen,
  .,
  Canadians,
  have,
  lost,
  more,
  than,
  $,
  1.2,
  million,
  to,
  COVID-19,
  scams,
  .,
  Úá,
  à,
  éó,
  ão,
  ção,
  üä,
  íï]]

## Tratamento para Números

É comum encontrarmos valores numéricos em meio ao texto. Em geral, tais informações não são consideradas relevantes - números podem ser muito frequentes - no entanto, em determinadas situações, manter os números no resultado final pode ser de extrema importância.

Veremos como tratar ambos os casos.

Se deseja-se remover os números, podemos iniciar convertendo os números que estão em forma escrita para algarismos arábicos (dígitos).

In [20]:
def word2number(tokens):
    _tokens = []
    for token in tokens:
        if token.pos_ == "NUM":
            continue
        else:
            _tokens.append(token)
    
    
    return _tokens


word2number(tokenize(["One president, twenty first century"])[0])

[president, ,, first, century]

Se deseja-se manter os números, convertê-los para um formato textual é uma decisão importante.

In [29]:
from num2words import num2words as n2w


def number2word(tokens):
    _tokens = []
    for token in tokens:
        if token.is_digit:
            print(token)
            _tokens.append(n2w(token.text))
        else:
            _tokens.append(token)
    
    
    return _tokens

print(n2w("1.2"))

number2word(tokenize(["1000 people, 1.2"])[0])

one point two
1000


['one thousand', people, ,, 1.2]

## Remoção de Stop Words

_Stop Words_ são palavras muito comuns e que não carregam (na maioria dos casos) significado próprio. Tais palavras podem enviezar os resultados de modelos, portanto seu tratamento é de extrema importância.

In [35]:
def remove_stop_words(tokens):
    _tokens = []
    for token in tokens:
        if not token.is_stop:
            _tokens.append(token)
            
    return _tokens

remove_stop_words(tokenize([document[:150]])[0])

[latest,
 :,
  ,
 lives,
 numbers,
 :,
 know,
 1,000,
 COVID-19,
 deaths,
 Canada,
 .,
 Health,
 expert,
 urges,
 provinces,
 ',
 find,
 mi]

## Lematização e _Stemming_

Os processos de lematização e de _stemming_ possuem o mesmo objetivo: reduzir formas flexionadas de linguagem (por exemplo verbos conjugados) para seu formato elementar. A diferença básica entre as técnicas é, sem muito rigor, que a lematização é um processo mais inteligente, capaz de encontrar os radicais mesmo para verbos irregulares (para o caso em que a palavra analisada é um verbo :)). Em geral, utilizar a lematização elimina a necessidade de _stemming_, e portanto utilizaremos apenas ela.

In [19]:
def lemmatize(document):
    lemmatized = []
    for token in document:
        lemmatized.append(token.lemma_)
        
    return lemmatized


lemmatize(tokenize([bad_txt])[0])

['<',
 'br><br',
 '>',
 'read',
 'the',
 'full',
 'story',
 ':',
 '<',
 'a',
 'href="https://t.co/2wagup1oc2">https://t.co/2wagup1oc2</a><a',
 'href="https://twitter.com',
 '/',
 'hashtag',
 '/',
 'Covid19nfld?src',
 '=',
 'hash&amp;ref_src',
 '=',
 'twsrc%5Etfw">#Covid19nfld</a',
 '>',
 '<',
 'a',
 'href="https://t.co',
 '/',
 'vi92xmwntv">pic.twitter.com',
 '/',
 'vi92xmwntv</a>&mdash;@cbcnl',
 'in',
 'Canada',
 "'s",
 'North',
 ',',
 '\xa0\xa0',
 'all',
 'of',
 'Yukon',
 "'s",
 '11',
 ' ',
 'the',
 'late',
 ':',
 ' ',
 'the',
 'life',
 'behind',
 'the',
 'number',
 ':',
 'what',
 '-PRON-',
 'know',
 'about',
 'the',
 'first',
 '1,000',
 'covid-19',
 'death',
 'in',
 'Canada',
 '.',
 'health',
 'expert',
 'urge',
 'province',
 'to',
 "'",
 'find',
 'the',
 'middle',
 'ground',
 "'",
 'as',
 '-PRON-',
 'begin',
 'to',
 'reopen',
 '.',
 'Canadians',
 'have',
 'lose',
 'more',
 'than',
 '$',
 '1.2',
 'million',
 'to',
 'COVID-19',
 'scam',
 '.',
 'Úá',
 'à',
 'éó',
 'ão',
 'ção',
 'üä',


## Exercício

Agora, você irá realizar o pré-processamento de texto em um conjunto de dados novo: o _[IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)_.

1. Baixe o arquivo localizado [neste link](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/download);
2. Inspecione e estudo o conteúdo do arquivo (isto pode ser feito utilizando a [página do _contest_ no Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews));
3. Construa um _pipeline_ de pré-processamento para o dataset.

**IMPORTANTE:** Ao construir seu _pipeline_, pense em quais informações seriam úteis (e quais não) para classificar uma avaliação como `positiva` ou `negativa`.

## Referências

[1] [All you need to know about text preprocessing for NLP and Machine Learning](https://www.kdnuggets.com/2019/04/text-preprocessing-nlp-machine-learning.html);

[2] [NLP Text Preprocessing: A Practical Guide and Template](https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79);